In [ ]:
!ls ~/my_virtualenvs


ls: cannot access '/home/samarth7022/my_virtualenvs': No such file or directory


In [ ]:
import tensorflow as tf

2024-11-10 15:38:38.212067: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-10 15:38:38.360298: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 15:38:38.411039: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 15:38:39.285969: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [ ]:
import tensorflow as tf

# List available GPUs
gpus = tf.config.list_physical_devices('GPU')

if len(gpus) > 1:
    # Set TensorFlow to only use GPU 1
    try:
        tf.config.set_visible_devices(gpus[1], 'GPU')
        # Optionally limit memory growth for GPU 1
        tf.config.experimental.set_memory_growth(gpus[1], True)
        print("GPU 1 is set for use.")
    except RuntimeError as e:
        print(e)
else:
    print("Less than two GPUs available.")


2024-11-11 16:56:45.231413: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-11 16:56:45.387939: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-11 16:56:45.439235: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-11 16:56:46.335543: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

GPU 1 is set for use.


In [ ]:
! nvidia-smi

Mon Nov 11 19:20:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.216.01             Driver Version: 535.216.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   49C    P0              82W / 300W |  81047MiB / 81920MiB |     35%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if len(gpus) > 0:
    # Set memory growth to avoid memory issues
    tf.config.experimental.set_memory_growth(gpus[0], True)



In [ ]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [ ]:
import pandas as pd
import numpy as np
import gc
import os
from typing import List, Dict, Iterable

# --- Constants ---
PATH_TO_DATA = "./Hydraulics/"
NUMBER_OF_PROFILES = 2205
PROFILE_MAX_SAMPLE_RATE = 6000
TARGET_NAMES = ["cooler", "valve", "leakage", "accumulator", "stable"]

# --- Sensor Files Configuration ---
sensor_files_config = [
    {"name": "CE", "upsample_coeff": 100},
    {"name": "CP", "upsample_coeff": 100},
    {"name": "EPS1", "upsample_coeff": 1},
    {"name": "FS1", "upsample_coeff": 10},
    {"name": "FS2", "upsample_coeff": 10},
    {"name": "PS1", "upsample_coeff": 1},
    {"name": "PS2", "upsample_coeff": 1},
    {"name": "PS3", "upsample_coeff": 1},
    {"name": "PS4", "upsample_coeff": 1},
    {"name": "PS5", "upsample_coeff": 1},
    {"name": "PS6", "upsample_coeff": 1},
    {"name": "SE", "upsample_coeff": 100},
    {"name": "TS1", "upsample_coeff": 100},
    {"name": "TS2", "upsample_coeff": 100},
    {"name": "TS3", "upsample_coeff": 100},
    {"name": "TS4", "upsample_coeff": 100},
    {"name": "VS1", "upsample_coeff": 100},
]

# --- Data Loading Functions ---
def get_files_with_resample(config: List[Dict]) -> Iterable[np.ndarray]:
    """
    Load sensor data files, resample based on the configuration, and return the data as a generator.
    """
    for file in config:
        try:
            file_path = os.path.join(PATH_TO_DATA, file["name"] + ".txt")
            data = np.genfromtxt(file_path, dtype=float, delimiter='\t')
            # Resample data if necessary
            yield np.repeat(data, file["upsample_coeff"], axis=1).flatten()
        except Exception as e:
            print(f"Error loading file {file['name']}: {e}")

def load_feature_dataframe(config: List[Dict]) -> pd.DataFrame:
    """
    Load feature data into a pandas DataFrame with profile IDs.
    """
    columns = [file["name"] for file in config]
    # Convert the generator into a list before stacking
    data = np.stack(list(get_files_with_resample(config)), axis=-1)  # Stack all data into a matrix
    data_df = pd.DataFrame(data, columns=columns)

    profile_ids = np.repeat(range(1, NUMBER_OF_PROFILES + 1), PROFILE_MAX_SAMPLE_RATE)
    profile_ids_df = pd.DataFrame(profile_ids, columns=["profile_id"])

    return pd.concat([profile_ids_df, data_df], axis=1, sort=False)


def load_targets(filename: str) -> pd.DataFrame:
    """
    Load the target conditions (e.g., stable, valve failure) for each profile.
    """
    try:
        conditions_data = np.genfromtxt(os.path.join(PATH_TO_DATA, filename), dtype=int, delimiter='\t')
        conditions_df = pd.DataFrame(conditions_data, columns=TARGET_NAMES)

        profile_ids = range(1, NUMBER_OF_PROFILES + 1)
        profile_ids_df = pd.DataFrame(profile_ids, columns=["profile_id"])

        return pd.concat([profile_ids_df, conditions_df], axis=1, sort=False)
    except Exception as e:
        print(f"Error loading target file: {e}")

# --- Load Features and Targets ---
feature_df = load_feature_dataframe(sensor_files_config)
target_df = load_targets("profile.txt")

# --- Memory Cleanup ---
_ = gc.collect()

# --- Merge Features and Targets by 'profile_id' ---
merged_df = pd.merge(feature_df, target_df, on="profile_id", how="inner")

# --- Handle Missing Values ---
merged_df.replace([np.inf, -np.inf], np.nan, inplace=True)
merged_df.dropna(axis=0, inplace=True)  # Drop rows with NaN values

# --- Show Sample Data ---
print(merged_df.head())

# --- Example of Data Preprocessing and Feature Selection ---
# Extract feature columns (excluding target columns)
X = merged_df.drop(columns=TARGET_NAMES)  # Drop all target columns, keep 'profile_id'
y = merged_df[["profile_id", "stable"]]  # Keep only 'profile_id' and 'stable' as target columns

# --- One-hot encode the target 'stable' column ---
y = pd.get_dummies(y["stable"], prefix="stable")

print(f"Features shape: {X.shape}, Target shape: {y.shape}")


   profile_id      CE     CP    EPS1   FS1     FS2     PS1     PS2    PS3  \
0           1  47.202  2.184  2411.6  8.99  10.179  151.47  125.50  2.305   
1           1  47.202  2.184  2411.6  8.99  10.179  151.45  125.39  2.305   
2           1  47.202  2.184  2411.6  8.99  10.179  151.52  125.40  2.336   
3           1  47.202  2.184  2411.6  8.99  10.179  151.27  125.03  2.578   
4           1  47.202  2.184  2411.6  8.99  10.179  150.80  124.05  2.977   

   PS4  ...    TS1     TS2    TS3     TS4    VS1  cooler  valve  leakage  \
0  0.0  ...  35.57  40.961  38.32  30.363  0.604       3    100        0   
1  0.0  ...  35.57  40.961  38.32  30.363  0.604       3    100        0   
2  0.0  ...  35.57  40.961  38.32  30.363  0.604       3    100        0   
3  0.0  ...  35.57  40.961  38.32  30.363  0.604       3    100        0   
4  0.0  ...  35.57  40.961  38.32  30.363  0.604       3    100        0   

   accumulator  stable  
0          130       1  
1          130       1  
2    

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# Ensure the profile split is stratified based on the stable flag
# Get a list of unique profile ids
profile_ids = merged_df['profile_id'].unique()

# Get the stable flag for each profile
profile_stable_flags = merged_df.groupby('profile_id')['stable'].first().values

# Perform stratified split based on profile_stable_flags
train_profile_ids, test_profile_ids = train_test_split(profile_ids,
                                                       test_size=0.2,  # 80% train, 20% test
                                                       stratify=profile_stable_flags)

# Now filter the main dataset to get the training and testing sets based on the split profile ids
train_df = merged_df[merged_df['profile_id'].isin(train_profile_ids)]
test_df = merged_df[merged_df['profile_id'].isin(test_profile_ids)]

# Check the distribution of 'stable' in train and test sets
print(f"Train set distribution:\n{train_df['stable'].value_counts()}")
print(f"Test set distribution:\n{test_df['stable'].value_counts()}")


Train set distribution:
stable
0    6954000
1    3630000
Name: count, dtype: int64
Test set distribution:
stable
0    1740000
1     906000
Name: count, dtype: int64


In [ ]:
PATH_TO_DATA = "./Hydraulics/"


In [ ]:
# --- Preprocessing for transformer model ---
# Separate features and targets for training and testing
X_train = train_df.drop(columns=["profile_id", "stable"])
y_train = train_df[["profile_id", "stable"]]

X_test = test_df.drop(columns=["profile_id", "stable"])
y_test = test_df[["profile_id", "stable"]]

# Sequence shaping for transformer model (reshape for temporal data)
X_train_seq = X_train.values.reshape(-1, PROFILE_MAX_SAMPLE_RATE, X_train.shape[1])
X_test_seq = X_test.values.reshape(-1, PROFILE_MAX_SAMPLE_RATE, X_test.shape[1])

# Ensure target y is in the correct format (for binary classification)
y_train = pd.get_dummies(y_train['stable'])
y_test = pd.get_dummies(y_test['stable'])

# --- Confirm final shapes ---
print(f"Training data shape: {X_train_seq.shape}, Training target shape: {y_train.shape}")
print(f"Testing data shape: {X_test_seq.shape}, Testing target shape: {y_test.shape}")


Training data shape: (1764, 6000, 21), Training target shape: (10584000, 2)
Testing data shape: (441, 6000, 21), Testing target shape: (2646000, 2)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.utils import Progbar
import matplotlib.pyplot as plt
from sklearn.svm import OneClassSVM

# --- Positional Encoding ---
def positional_encoding(position, d_model):
    angle_rads = tf.range(start=0, limit=position, delta=1, dtype=tf.float32)
    angle_rads = angle_rads[:, tf.newaxis]  # Shape: (position, 1)

    i = tf.range(d_model, dtype=tf.float32)
    i = i[tf.newaxis, :]  # Shape: (1, d_model)

    angle_rates = 1 / tf.pow(10000.0, (2 * (i // 2)) / tf.cast(d_model, dtype=tf.float32))
    angle_rads = angle_rads * angle_rates  # Shape: (position, d_model)

    sines = tf.sin(angle_rads[:, 0::2])  # Apply sine to even indices
    cosines = tf.cos(angle_rads[:, 1::2])  # Apply cosine to odd indices

    pos_encoding = tf.concat([sines, cosines], axis=-1)  # Shape: (position, d_model)

    return pos_encoding

# --- Transformer Autoencoder Model ---
def create_transformer_autoencoder(input_shape, d_model=21):
    inputs = layers.Input(shape=input_shape)

    # Positional Encoding
    pos_enc = positional_encoding(input_shape[0], d_model)  # Shape: (seq_len, d_model)
    pos_enc = tf.expand_dims(pos_enc, axis=0)  # Add batch dimension, shape: (1, seq_len, d_model)
    pos_enc = tf.tile(pos_enc, [tf.shape(inputs)[0], 1, 1])  # Tile to match batch size, shape: (batch_size, seq_len, d_model)

    inputs_with_pos = inputs + pos_enc

    # Encoder
    x = layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(inputs_with_pos)  # Conv1D
    x = layers.MultiHeadAttention(num_heads=4, key_dim=32)(x, x)  # Multi-Head Attention
    x = layers.LayerNormalization()(x)
    x = layers.Dropout(0.1)(x)

    # Match the channels of inputs_with_pos to x before residual connection
    inputs_with_pos_resized = layers.Conv1D(filters=32, kernel_size=1, padding='same')(inputs_with_pos)

    # Residual connection with matching channels
    x = layers.add([x, inputs_with_pos_resized])  # Residual connection
    x = layers.GlobalAveragePooling1D()(x)

    # Decoder
    x = layers.RepeatVector(input_shape[0])(x)
    x = layers.LSTM(128, return_sequences=True)(x)  # LSTM
    x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)  # MLP for reconstruction
    x = layers.TimeDistributed(layers.Dense(32, activation='relu'))(x)
    decoded = layers.TimeDistributed(layers.Dense(input_shape[1]))(x)

    # Model setup
    model = models.Model(inputs, decoded)
    return model


# --- Training and Anomaly Detection ---
# (Assuming you have your X_train_seq, X_test_seq, y_train, y_test data loaded)

# Create the model
input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])  # Adjust shape according to your data
model = create_transformer_autoencoder(input_shape)

# Compile the model (with a smaller learning rate to help with stability)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')

# --- Memory Efficient Training (with Batch Generator) ---
def batch_generator(data, batch_size):
    """Generates batches of data for training."""
    num_samples = data.shape[0]
    for i in range(0, num_samples, batch_size):
        yield data[i:i + batch_size]



# --- Training Loop with Progbar ---
epochs = 10
batch_size = 8
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")

    progbar = Progbar(target=len(X_train_seq)//batch_size)

    for batch_idx, batch_x in enumerate(batch_generator(X_train_seq, batch_size)):
        loss = model.train_on_batch(batch_x, batch_x)

        # Update progress bar with dynamic loss value
        progbar.update(batch_idx + 1, values=[("loss", loss)])

    # Save the model after each epoch
    model.save('my_model_epoch_{}.h5'.format(epoch+1))
    print(f"Model saved at epoch {epoch+1}")

# --- Save the final model after training ---
model.save('final_model.h5')  # Save the final model after all epochs are completed
print("Final model saved successfully!")



2024-11-10 16:40:07.157975: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-10 16:40:07.269835: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 16:40:07.299150: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 16:40:07.816468: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

Epoch 1/10


2024-11-10 16:40:15.009685: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8905
2024-11-10 16:40:15.178143: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda-11.2/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget


221/220 [==============================] - 71s 305ms/step - loss: 264616.2033
Model saved at epoch 1
Epoch 2/10
221/220 [==============================] - 60s 270ms/step - loss: 52222.0708
Model saved at epoch 2
Epoch 3/10
221/220 [==============================] - 59s 269ms/step - loss: 2442.1481
Model saved at epoch 3
Epoch 4/10
221/220 [==============================] - 61s 276ms/step - loss: 2488.5415
Model saved at epoch 4
Epoch 5/10
221/220 [==============================] - 76s 344ms/step - loss: 2490.5553
Model saved at epoch 5
Epoch 6/10
221/220 [==============================] - 72s 327ms/step - loss: 2491.1782
Model saved at epoch 6
Epoch 7/10
221/220 [==============================] - 60s 271ms/step - loss: 2490.9787
Model saved at epoch 7
Epoch 8/10
221/220 [==============================] - 59s 269ms/step - loss: 2490.1199
Model saved at epoch 8
Epoch 9/10
221/220 [==============================] - 59s 268ms/step - loss: 2490.8232
Model saved at epoch 9
Epoch 10/10
221/22

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('final_model.h5')
print("Model loaded successfully!")

2024-11-10 17:32:59.401587: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-10 17:32:59.560307: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 17:32:59.609144: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 17:33:01.189070: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

Model loaded successfully!


In [ ]:
import tensorflow as tf
import numpy as np

# Custom callback to print the GPU used after each epoch
class PrintGPUUsageCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Get the current device being used
        device = tf.config.experimental.list_physical_devices('GPU')[gpu_number]
        print(f"Epoch {epoch + 1} completed. Using {device.name}.")

gpu_number = 1  # Use the second GPU (GPU 1)
gpus = tf.config.list_physical_devices('GPU')

# Set the second GPU as visible
if gpus:
    if gpu_number < len(gpus):
        tf.config.experimental.set_visible_devices(gpus[gpu_number], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[gpu_number], True)

        # Verify GPU usage
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPU(s)")

        # Sample model definition
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10, activation='softmax')
        ])

        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        # Create random training data
        x_train = np.random.random((1000, 784))
        y_train = np.random.randint(0, 10, size=(1000,))

        # Train the model using GPU 1
        with tf.device('/GPU:1'):
            model.fit(x_train, y_train, epochs=5, batch_size=32, callbacks=[PrintGPUUsageCallback()])
    else:
        print(f"GPU number {gpu_number} is out of range. Available GPUs are indexed from 0 to {len(gpus)-1}.")
else:
    print("No GPUs available.")


2 Physical GPUs, 1 Logical GPU(s)
Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 2.4289 - accuracy: 0.1100
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 2.3014 - accuracy: 0.1260
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 2.2840 - accuracy: 0.1220
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 2.2732 - accuracy: 0.1370
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 2.2386 - accuracy: 0.1660


In [ ]:
train_reconstruction = model.predict(X_train_seq)
test_reconstruction = model.predict(X_test_seq)

In [ ]:

# --- Loading the Model for Prediction ---
# After training, you can load the model for prediction later
# Uncomment the following lines to load the model (if needed)


# --- Anomaly Detection ---
train_reconstruction = model.predict(X_train_seq)
test_reconstruction = model.predict(X_test_seq)

# Calculate MSE
train_mse = np.mean(np.square(X_train_seq - train_reconstruction), axis=(1, 2))
test_mse = np.mean(np.square(X_test_seq - test_reconstruction), axis=(1, 2))

# Adaptive Thresholding with One-Class SVM
ocsvm = OneClassSVM(nu=0.01, kernel='rbf', gamma='scale')
ocsvm.fit(train_mse.reshape(-1, 1))

# Detect anomalies
train_anomalies = ocsvm.predict(train_mse.reshape(-1, 1)) == -1
test_anomalies = ocsvm.predict(test_mse.reshape(-1, 1)) == -1

# --- Accuracy Calculation ---
y_train_stable = y_train["stable_0"].values  # Ensure this column exists in y_train
y_test_stable = y_test["stable_0"].values

train_anomalies_binary = train_anomalies.astype(int)
test_anomalies_binary = test_anomalies.astype(int)

train_accuracy = np.mean(train_anomalies_binary == y_train_stable)
test_accuracy = np.mean(test_anomalies_binary == y_test_stable)

print(f"Training set accuracy: {train_accuracy:.4f}")
print(f"Test set accuracy: {test_accuracy:.4f}")

# --- Visualization (Optional) ---
plt.plot(train_mse[:100], label="Train MSE (first 100 samples)")
plt.axhline(y=ocsvm.decision_function(train_mse.reshape(-1, 1)).mean(), color='r', linestyle='--', label="Threshold")
plt.legend()
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.utils import Progbar
import matplotlib.pyplot as plt
from sklearn.svm import OneClassSVM

# --- Positional Encoding ---
def positional_encoding(position, d_model):
    angle_rads = tf.range(start=0, limit=position, delta=1, dtype=tf.float32)
    angle_rads = angle_rads[:, tf.newaxis]  # Shape: (position, 1)

    i = tf.range(d_model, dtype=tf.float32)
    i = i[tf.newaxis, :]  # Shape: (1, d_model)

    angle_rates = 1 / tf.pow(10000.0, (2 * (i // 2)) / tf.cast(d_model, dtype=tf.float32))
    angle_rads = angle_rads * angle_rates  # Shape: (position, d_model)

    sines = tf.sin(angle_rads[:, 0::2])  # Apply sine to even indices
    cosines = tf.cos(angle_rads[:, 1::2])  # Apply cosine to odd indices

    pos_encoding = tf.concat([sines, cosines], axis=-1)  # Shape: (position, d_model)

    return pos_encoding

# --- Transformer Autoencoder Model ---
# --- Transformer Autoencoder Model ---
def create_transformer_autoencoder(input_shape, d_model=21):
    inputs = layers.Input(shape=input_shape)

    # Positional Encoding
    pos_enc = positional_encoding(input_shape[0], d_model)  # Shape: (seq_len, d_model)
    pos_enc = tf.expand_dims(pos_enc, axis=0)  # Add batch dimension, shape: (1, seq_len, d_model)
    pos_enc = tf.tile(pos_enc, [tf.shape(inputs)[0], 1, 1])  # Tile to match batch size, shape: (batch_size, seq_len, d_model)

    inputs_with_pos = inputs + pos_enc

    # Encoder
    x = layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(inputs_with_pos)  # Conv1D
    x = layers.MultiHeadAttention(num_heads=4, key_dim=32)(x, x)  # Multi-Head Attention
    x = layers.LayerNormalization()(x)
    x = layers.Dropout(0.1)(x)

    # Match the channels of inputs_with_pos to x before residual connection
    inputs_with_pos_resized = layers.Conv1D(filters=32, kernel_size=1, padding='same')(inputs_with_pos)

    # Residual connection with matching channels
    x = layers.add([x, inputs_with_pos_resized])  # Residual connection
    x = layers.GlobalAveragePooling1D()(x)

    # Decoder
    x = layers.RepeatVector(input_shape[0])(x)
    x = layers.LSTM(128, return_sequences=True)(x)  # LSTM
    x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)  # MLP for reconstruction
    x = layers.TimeDistributed(layers.Dense(32, activation='relu'))(x)
    decoded = layers.TimeDistributed(layers.Dense(input_shape[1]))(x)

    # Model setup
    model = models.Model(inputs, decoded)
    return model



# --- Training and Anomaly Detection ---
# (Assuming you have your X_train_seq, X_test_seq, y_train, y_test data loaded)

# Create the model
input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])  # Adjust shape according to your data
model = create_transformer_autoencoder(input_shape)

# Compile the model (with a smaller learning rate to help with stability)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')

# --- Memory Efficient Training (with Batch Generator) ---
def batch_generator(data, batch_size):
    """Generates batches of data for training."""
    num_samples = data.shape[0]
    for i in range(0, num_samples, batch_size):
        yield data[i:i + batch_size]



# --- Training Loop with Progbar ---
epochs = 10
batch_size = 8
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")

    progbar = Progbar(target=len(X_train_seq)//batch_size)

    for batch_idx, batch_x in enumerate(batch_generator(X_train_seq, batch_size)):
        loss = model.train_on_batch(batch_x, batch_x)

        # Update progress bar with dynamic loss value
        progbar.update(batch_idx + 1, values=[("loss", loss)])


# --- Anomaly Detection ---
train_reconstruction = model.predict(X_train_seq)
test_reconstruction = model.predict(X_test_seq)

# Calculate MSE
train_mse = np.mean(np.square(X_train_seq - train_reconstruction), axis=(1, 2))
test_mse = np.mean(np.square(X_test_seq - test_reconstruction), axis=(1, 2))

# Adaptive Thresholding with One-Class SVM
ocsvm = OneClassSVM(nu=0.01, kernel='rbf', gamma='scale')
ocsvm.fit(train_mse.reshape(-1, 1))

# Detect anomalies
train_anomalies = ocsvm.predict(train_mse.reshape(-1, 1)) == -1
test_anomalies = ocsvm.predict(test_mse.reshape(-1, 1)) == -1

# --- Accuracy Calculation ---
y_train_stable = y_train["stable_0"].values  # Ensure this column exists in y_train
y_test_stable = y_test["stable_0"].values

train_anomalies_binary = train_anomalies.astype(int)
test_anomalies_binary = test_anomalies.astype(int)

train_accuracy = np.mean(train_anomalies_binary == y_train_stable)
test_accuracy = np.mean(test_anomalies_binary == y_test_stable)

print(f"Training set accuracy: {train_accuracy:.4f}")
print(f"Test set accuracy: {test_accuracy:.4f}")

# --- Visualization (Optional) ---
plt.plot(train_mse[:100], label="Train MSE (first 100 samples)")
plt.axhline(y=ocsvm.decision_function(train_mse.reshape(-1, 1)).mean(), color='r', linestyle='--', label="Threshold")
plt.legend()
plt.show()


Epoch 1/10
221/220 [==============================] - 63s 275ms/step - loss: 266689.8339
Epoch 2/10
221/220 [==============================] - 69s 313ms/step - loss: 45773.9149
Epoch 3/10
221/220 [==============================] - 75s 338ms/step - loss: 2441.3314
Epoch 4/10
221/220 [==============================] - 72s 326ms/step - loss: 2469.2652
Epoch 5/10
221/220 [==============================] - 70s 317ms/step - loss: 2470.4368
Epoch 6/10
221/220 [==============================] - 70s 318ms/step - loss: 2470.1669
Epoch 7/10
221/220 [==============================] - 73s 331ms/step - loss: 2469.2394
Epoch 8/10
221/220 [==============================] - 72s 327ms/step - loss: 2466.9887
Epoch 9/10
221/220 [==============================] - 70s 316ms/step - loss: 2465.2994
Epoch 10/10
221/220 [==============================] - 73s 328ms/step - loss: 2464.0441


In [ ]:
# --- Anomaly Detection ---
# After training the model, calculate the reconstruction error on both training and test sets.
train_reconstruction = model.predict(X_train_seq)
test_reconstruction = model.predict(X_test_seq)

# Calculate Mean Squared Error (MSE) between input and output (reconstructed data)
train_mse = np.mean(np.square(X_train_seq - train_reconstruction), axis=(1, 2))  # MSE per sample
test_mse = np.mean(np.square(X_test_seq - test_reconstruction), axis=(1, 2))

# Threshold for anomaly detection (can be tuned based on the distribution of MSE values)
threshold = np.percentile(train_mse, 95)  # Define a threshold (e.g., 95th percentile of MSE)

# Detect anomalies based on the threshold
train_anomalies = train_mse > threshold
test_anomalies = test_mse > threshold

# Print anomaly counts
print(f"Training set anomalies: {np.sum(train_anomalies)}")
print(f"Test set anomalies: {np.sum(test_anomalies)}")

# --- Accuracy Calculation ---
# Assuming that the ground truth for anomalies is given by the "stable" flag (0 = stable, 1 = unstable)
# We assume stable is "normal" and unstable as "anomaly"
y_train_stable = y_train["stable_0"].values  # If 0 is stable, then `stable_0` column should be 1 for stable
y_test_stable = y_test["stable_0"].values

# Convert anomaly detection result to binary (0 = no anomaly, 1 = anomaly)
train_anomalies_binary = train_anomalies.astype(int)
test_anomalies_binary = test_anomalies.astype(int)

# Calculate accuracy by comparing predicted anomalies to actual anomalies (stable = 0, anomaly = 1)
train_accuracy = np.mean(train_anomalies_binary == y_train_stable)
test_accuracy = np.mean(test_anomalies_binary == y_test_stable)

print(f"Training set accuracy: {train_accuracy:.4f}")
print(f"Test set accuracy: {test_accuracy:.4f}")

# Optionally, you can visualize some of the anomalies
plt.plot(train_mse[:100], label="Train MSE (first 100 samples)")
plt.axhline(y=threshold, color='r', linestyle='--', label="Threshold")
plt.legend()
plt.show()

In [ ]:

# --- Anomaly Detection ---
# After training the model, calculate the reconstruction error on both training and test sets.
train_reconstruction = model.predict(X_train_seq)
test_reconstruction = model.predict(X_test_seq)

# Calculate Mean Squared Error (MSE) between input and output (reconstructed data)
train_mse = np.mean(np.square(X_train_seq - train_reconstruction), axis=(1, 2))  # MSE per sample
test_mse = np.mean(np.square(X_test_seq - test_reconstruction), axis=(1, 2))

# Threshold for anomaly detection (can be tuned based on the distribution of MSE values)
threshold = np.percentile(train_mse, 95)  # Define a threshold (e.g., 95th percentile of MSE)

# Detect anomalies based on the threshold
train_anomalies = train_mse > threshold
test_anomalies = test_mse > threshold

# Print anomaly counts
print(f"Training set anomalies: {np.sum(train_anomalies)}")
print(f"Test set anomalies: {np.sum(test_anomalies)}")

# --- Accuracy Calculation ---
# Assuming that the ground truth for anomalies is given by the "stable" flag (0 = stable, 1 = unstable)
# We assume stable is "normal" and unstable as "anomaly"
y_train_stable = y_train["stable_0"].values  # If 0 is stable, then `stable_0` column should be 1 for stable
y_test_stable = y_test["stable_0"].values

# Convert anomaly detection result to binary (0 = no anomaly, 1 = anomaly)
train_anomalies_binary = train_anomalies.astype(int)
test_anomalies_binary = test_anomalies.astype(int)

# Calculate accuracy by comparing predicted anomalies to actual anomalies (stable = 0, anomaly = 1)
train_accuracy = np.mean(train_anomalies_binary == y_train_stable)
test_accuracy = np.mean(test_anomalies_binary == y_test_stable)

print(f"Training set accuracy: {train_accuracy:.4f}")
print(f"Test set accuracy: {test_accuracy:.4f}")

# Optionally, you can visualize some of the anomalies
plt.plot(train_mse[:100], label="Train MSE (first 100 samples)")
plt.axhline(y=threshold, color='r', linestyle='--', label="Threshold")
plt.legend()
plt.show()


NameError: name 'model' is not defined

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

# 1. Set up MirroredStrategy for Multi-GPU
strategy = tf.distribute.MirroredStrategy()

print('Number of devices: ', strategy.num_replicas_in_sync)

# 2. Define the model inside the strategy scope
with strategy.scope():
    # Transformer Autoencoder Model for Anomaly Detection
    def create_transformer_autoencoder(input_shape):
        inputs = layers.Input(shape=input_shape)

        # Encoder
        x = layers.MultiHeadAttention(num_heads=8, key_dim=64)(inputs, inputs)
        x = layers.LayerNormalization()(x)
        x = layers.Dropout(0.1)(x)  # Dropout to reduce overfitting
        x = layers.GlobalAveragePooling1D()(x)  # Pooling to reduce dimensionality

        # Decoder (Symmetric to Encoder)
        x = layers.RepeatVector(input_shape[0])(x)  # Repeat to match the input shape
        x = layers.LSTM(128, return_sequences=True)(x)
        x = layers.TimeDistributed(layers.Dense(64))(x)
        decoded = layers.TimeDistributed(layers.Dense(input_shape[1]))(x)  # Output shape should match input shape

        # Model setup
        model = models.Model(inputs, decoded)
        return model

    # 3. Create the Transformer Autoencoder model
    input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])  # shape: (timesteps, features)
    model = create_transformer_autoencoder(input_shape)

    # 4. Compile the model with a reconstruction error loss
    model.compile(optimizer='adam', loss='mean_squared_error')

# 5. Model Summary
model.summary()

# Check the number of GPUs available
physical_devices = tf.config.list_physical_devices('GPU')
print(f"Num GPUs Available: {len(physical_devices)}")

# Enable logging of device placement
tf.debugging.set_log_device_placement(True)

# Set memory growth to avoid out of memory issues
if physical_devices:
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)

# 6. Train the model using the MirroredStrategy
history = model.fit(
    X_train_seq, X_train_seq,
    epochs=10,
    batch_size=32,  # Batch size for each GPU
    validation_data=(X_test_seq, X_test_seq),
    verbose=0
)

# 7. Plotting the loss curves
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

# Enable mixed precision training (optional for performance)
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# 1. Use MirroredStrategy for Multi-GPU training
strategy = tf.distribute.MirroredStrategy()

# 2. Load CIFAR-10 dataset for demonstration
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

# 3. Define a Convolutional Neural Network (CNN) Model
def create_cnn_model():
    model = models.Sequential([
        layers.InputLayer(input_shape=(32, 32, 3)),  # CIFAR-10 images are 32x32x3
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')  # 10 classes for CIFAR-10
    ])
    return model

# 4. Define and compile the model within the strategy scope
with strategy.scope():
    model = create_cnn_model()
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# 5. Print the model summary
model.summary()

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_log_device_placement(True)
        # Limit GPU memory growth
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


# 6. Train the model using MirroredStrategy
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=16,  # Try reducing the batch size
    validation_data=(x_test, y_test),
    verbose=2
)

# 7. Plot training and validation accuracy and loss
plt.figure(figsize=(12, 6))

# Training accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Training loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# 8. Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPUs will likely run quickly with dtype policy mixed_float16 as they all have compute capability of at least 7.0
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


2024-11-10 15:35:13.862593: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-10 15:35:14.843911: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78910 MB memory:  -> device: 1, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:31:00.0, compute capability: 8.0


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                        

AttributeError: module 'tensorflow._api.v2.config' has no attribute 'set_log_device_placement'

In [ ]:
import tensorflow as tf

# Check the number of GPUs available
physical_devices = tf.config.list_physical_devices('GPU')
print(f"Num GPUs Available: {len(physical_devices)}")

# Enable logging of device placement
tf.debugging.set_log_device_placement(True)

# Set memory growth to avoid out of memory issues
if physical_devices:
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)

# Now, you can proceed to train your model
# history = model.fit(...)  # Your model fitting code here


Num GPUs Available: 2


In [ ]:
import tensorflow as tf

# List available GPUs
gpus = tf.config.list_physical_devices('GPU')

# Ensure that GPU 1 exists
if len(gpus) > 1:
    # Set memory growth to avoid OOM errors
    tf.config.set_visible_devices(gpus[1], 'GPU')  # Only use GPU 1
    tf.config.experimental.set_memory_growth(gpus[1], True)
else:
    print("GPU 1 is not available.")


2024-11-10 15:34:06.069917: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-10 15:34:06.222510: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 15:34:06.282504: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 15:34:07.088237: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [ ]:
import tensorflow as tf


model = tf.keras.models.load_model('final_model.h5')
print("Model loaded successfully!")

Model loaded successfully!


In [ ]:
train_reconstruction = model.predict(X_train_seq)
test_reconstruction = model.predict(X_test_seq)

2024-11-11 11:58:42.313053: E tensorflow/stream_executor/cuda/cuda_dnn.cc:389] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2024-11-11 11:58:42.313132: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at conv_ops.cc:1134 : UNIMPLEMENTED: DNN library is not found.


UnimplementedError: Graph execution error:

Detected at node 'model/conv1d/Conv1D' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1046, in launch_instance
      app.start()
    File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
      self._run_once()
    File "/usr/lib/python3.10/asyncio/base_ev

In [ ]:
import tensorflow as tf
import time

# Check available GPUs
gpus = tf.config.list_physical_devices('GPU')
print(f"Available GPUs: {len(gpus)}")

if len(gpus) < 2:
    raise RuntimeError("This script requires at least 2 GPUs. Please check your hardware setup.")

# Set memory growth for both GPUs to avoid TensorFlow allocating all memory at once
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Set a memory limit for each GPU (e.g., 10GB per GPU) - for TensorFlow versions < 2.4
if len(gpus) >= 2:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10240)])  # 10GB for GPU-0
    tf.config.experimental.set_virtual_device_configuration(
        gpus[1], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10240)])  # 10GB for GPU-1

# Define a function to perform a heavy operation on a given GPU
def stress_test_gpu(device, size=1000):  # Reduced tensor size for safety
    with tf.device(device):
        print(f"Starting stress test on {device}...")

        # Create large random tensors
        A = tf.random.normal([size, size])
        B = tf.random.normal([size, size])

        # Perform a matrix multiplication operation repeatedly to stress the GPU
        start_time = time.time()
        for _ in range(40):  # Repeat 10 times
            C = tf.matmul(A, B)  # Matrix multiplication
            tf.reduce_sum(C)  # To ensure the result is actually computed
        end_time = time.time()

        print(f"Stress test on {device} complete in {end_time - start_time:.2f} seconds.")

# Run the stress tests sequentially (one GPU at a time)
def run_stress_tests():
    # First, stress GPU-0
    stress_test_gpu('/GPU:0', size=1000)  # Adjust tensor size to fit in memory

    # Then, stress GPU-1
    stress_test_gpu('/GPU:1', size=1000)  # Adjust tensor size to fit in memory

if __name__ == "__main__":
    run_stress_tests()


Available GPUs: 2


ValueError: Cannot set memory growth on device when virtual devices configured

In [ ]:
# List all available kernels
!jupyter kernelspec list


Available kernels:
  my_kernel        /home/samarth7022/.local/share/jupyter/kernels/my_kernel
  mobilevit        /usr/local/share/jupyter/kernels/mobilevit
  myenv            /usr/local/share/jupyter/kernels/myenv
  python3          /usr/local/share/jupyter/kernels/python3
  python3.10       /usr/local/share/jupyter/kernels/python3.10
  tenenv           /usr/local/share/jupyter/kernels/tenenv
  tensorflow2_5    /usr/local/share/jupyter/kernels/tensorflow2_5
  unet             /usr/local/share/jupyter/kernels/unet
